In [1]:
"""This jupyter notebook section provides a sample usage of the protocol."""

import math
import plotly.figure_factory as ff
import content.helper.helper as helper
from plotly.offline import init_notebook_mode, iplot
from content.encryption.encryption import Encryption
from content.analyzers.key_analyzer import KeyAnalyzer

# Init the notebook for usage of Plotly.
init_notebook_mode(connected=True)

# Ask user for a message to encrypt and the desired cube side length.
message = input("Please type in the message you want to encrypt.\n")
cube_side_length = int(input("\nPlease type in the desired cube side length.\n"))

# Initialize the protocol.
protocol = Encryption(message=message, cube_side_length=cube_side_length)

# Print the layout of the cube.
print("\nThe layout of the desired cube is: ")
print(helper.get_cube_layout(cube_side_length=cube_side_length))

# Ask the user for a desired key length and then generate the key.
key_length = int(input("\nPlease type in the desired key length.\n"))
key = helper.generate_random_keys(
    length=key_length, max_index=math.floor(cube_side_length / 2)
)

# Print the random generated key.
print("\nThe random generated key is: ")
iplot(ff.create_table(helper.get_key_table(key=key)))

# Perform the key analyzing and print the new key again.
key = KeyAnalyzer(key=key).analyze()
print(f"\nThe reduced key has length of {len(key)}, and it is: ")
iplot(ff.create_table(helper.get_key_table(key=key)))

# Show user the current binary (The one after XOR operation).
print(f"\nThe plain-text in binary padded is: \n{protocol.get_current_binary()}\n")

# Encrypt the message based on the given key and show the encrypted binary.
protocol.encrypt(key=key)
print(f"\nThe cipher-text in binary is: \n{protocol.get_current_binary()}\n")

# Decrypt the message and show the plain-text recovered.
print(f"\nThe decrypted message is: {protocol.get_decrypted_str()}")


Please type in the message you want to encrypt.
Hi, this is Weiqi speaking!

Please type in the desired cube side length.
3

The layout of the desired cube is: 
    L1  C  R1
T1   0  0   0
C    0  0   0
D1   0  0   0

Please type in the desired key length.
25

The random generated key is: 



The reduced key has length of 20, and it is: 



The plain-text in binary padded is: 
101010000110010101011001001110110001111000001011110111110010100101110111010111111000110111000011010100101100111000000000110001110101000110110110101001100011101101100101101001010111001101101111111011100011000001011010001111010000101011001001111101100111100001110011010011011000000111111011000010010001000101011111110101110010011010111001110111011111011001000111111101000100010010111101011111101101101111111000011110111001101001010011010011001000111100011000101001100111101111111110110100001110000010000111110010000000110111100010111100110000000011001000111100011000101001100111101111111110110100001110000010000111110010000000110111100010111100110000


The cipher-text in binary is: 
1000100111100001100010011110001011111111010101111101101001000100010010111110001101100011110110010101101011010000000101110110011011001011110001000100100100111111111100011000100011101010001011010010101110101010111100111001011110001101111010110111000011001000011001101101111011111010

In [2]:
"""Analyze the reduced length of random generated keys."""

import numpy as np
from content.helper.helper import generate_random_keys
from content.analyzers.key_analyzer import KeyAnalyzer
# ------------------------- Results -----------------------------
# When key length is 20, the reduced key length is around: 15.05.
# When key length is 21, the reduced key length is around: 15.79.
# When key length is 25, the reduced key length is around: 18.73.
# When key length is 26, the reduced key length is around: 19.50.
# When key length is 27, the reduced key length is around: 20.20.
# When key length is 28, the reduced key length is around: 20.90.

# ------------------- The analyze process -----------------------
# Find the reduced key length in average.
length = 27
key_length = [
    len(KeyAnalyzer(key=generate_random_keys(length=length, max_index=1)).analyze())
    for _ in range(1000)
]
# Print the mean.
print(f"Key with length {length} will be reduced to length of {np.mean(key_length)}.")


Key with length 27 will be reduced to length of 20.395.


In [3]:
"""Analyze the reduced length of random generated keys."""

from itertools import chain
from content.analyzers.cubie_location_analyzer import CubieLocationAnalyzer

# Initialize the location analyzer with desired cube size and track location.
step = 5
location = 1
init_analyzer = CubieLocationAnalyzer(
    cube_side_length=3, track_item_location=location
)

# Get the initial locations.
locations = init_analyzer.get_all_location()

# Keep expanding the locations for more steps.
for _ in range(step - 1):
    locations_lists = [
        CubieLocationAnalyzer(
            cube_side_length=3, 
            track_item_location=location
        ).get_all_location()
        for location in locations
    ]
    
    # Make the list a set to remove duplicates.
    locations = set(chain.from_iterable(locations_lists))

print(f"Bit at location {location} can reach {len(locations)} different locations after {step} steps.")


Bit at location 1 can reach 216 different locations after 5 steps.


In [4]:
"""Track location of a certain bit after a list of moves."""

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from content.helper.helper import generate_random_keys
from content.analyzers.cubie_location_analyzer import CubieLocationAnalyzer

# Init the notebook for usage of Plotly.
init_notebook_mode(connected=True)

# Setup the analyzer and keys.
analyzer = CubieLocationAnalyzer(cube_side_length=3, track_item_location=0)
keys = generate_random_keys(length=40, max_index=1)

# Get locations and create the trace for scatter plot.
locations = analyzer.location_tracker(keys=keys)
trace = go.Scatter(
    x = [index + 1 for index in range(len(locations))],
    y = locations,
    mode = 'lines+markers'
)

# Plot the scatter plot in the notebook.
iplot([trace])


In [5]:
"""Track location of two bits after a list of moves."""

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from content.helper.helper import generate_random_keys
from content.analyzers.cubie_location_analyzer import CubieLocationAnalyzer

# Init the notebook for usage of Plotly.
init_notebook_mode(connected=True)

# Setup the analyzers and keys.
analyzer_one = CubieLocationAnalyzer(cube_side_length=3, track_item_location=0)
analyzer_two = CubieLocationAnalyzer(cube_side_length=3, track_item_location=1)
keys = generate_random_keys(length=20, max_index=1)

# Get locations and create the traces for scatter plot.
locations_one = analyzer_one.location_tracker(keys=keys)
locations_two = analyzer_two.location_tracker(keys=keys)

# Create a trace
trace1 = go.Scatter(
    x = [index + 1 for index in range(len(locations_one))],
    y = locations_one,
    mode = 'lines+markers'
)

# Create a trace
trace2 = go.Scatter(
    x = [index + 1 for index in range(len(locations_two))],
    y = locations_two,
    mode = 'lines+markers'
)

# Plot the scatter plot in the notebook.
iplot([trace1, trace2])

In [6]:
import numpy as np
from content.encryption.encryption import Encryption

# Set up the dummy message and side length.
cube_side_length = 3
dummy_message = "dummy_message"

# Function to get number of different bits.
def get_difference():
    binary_one = Encryption(
        message=dummy_message, cube_side_length=cube_side_length
    ).get_current_binary()

    binary_two = Encryption(
        message=dummy_message, cube_side_length=cube_side_length
    ).get_current_binary()
    
    return sum([1 if bit == binary_two[index] else 0 for index, bit in enumerate(binary_one)])

# Get list of number of different bits.
difference_list = [get_difference() for _ in range(1000)]

# Calculate the percentage and output message.
percent = round(np.mean(difference_list) / 216 * 100, 2)
print(f"On average, {percent}% bits will be different for the same input message when encrypted twice.")


On average, 49.87% bits will be different for the same input message when encrypted twice.
